In [4]:
import numpy as np
import pandas as pd
import os
positions = ['qb', 'rb', 'wr', 'te', 'k', 'dst']

In [5]:
master_df = pd.DataFrame(columns=["Player", "FPTS", "Position"])

for pos in positions:
    df = pd.read_csv('../2021/old_' + pos + '_cleaned.csv')
    # add a column to df that is the player's position
    df['Position'] = pos
    # add df to master_df
    master_df = pd.concat([master_df, df]).reset_index(drop=True)
    
# drop all NA rows for all dfs
master_df.dropna(inplace=True)

In [17]:
# set prices

means = [master_df.loc[master_df["Position"] == pos]["FPTS"].mean() for pos in positions]
stdevs = [master_df.loc[master_df["Position"] == pos]["FPTS"].std() for pos in positions]

# create a column for price and give each player a price that is their FPTS - mean / stdev
# for pos in positions:
#     master_df.loc[master_df["Position"] == pos, "Price"] = max((master_df.loc[master_df["Position"] == pos, "FPTS"] - means[positions.index(pos)]) / stdevs[positions.index(pos)] + 0.8, 0)

for pos in positions:
    price_series = (master_df.loc[master_df["Position"] == pos, "FPTS"] - means[positions.index(pos)]) / stdevs[positions.index(pos)] + 0.8
    master_df.loc[master_df["Position"] == pos, "Price"] = price_series.apply(lambda x: max(x, 0))

# truncate all prices to 2 decimal places
master_df["Price"] = master_df["Price"].apply(lambda x: round(x, 2))

# show all rows
pd.set_option('display.max_rows', None)
master_df.head(10)

,Player,FPTS,Position,Price
0,Josh Allen,417.7,qb,3.18
1,Justin Herbert,395.6,qb,3.01
2,Tom Brady,386.7,qb,2.94
3,Patrick Mahomes II,374.2,qb,2.84
4,Matthew Stafford,346.8,qb,2.63
5,Aaron Rodgers,336.3,qb,2.54
6,Dak Prescott,330.4,qb,2.50
7,Joe Burrow,328.1,qb,2.48
8,Jalen Hurts,321.2,qb,2.43
9,Kyler Murray,310.5,qb,2.34


In [18]:
# write out master_df
master_df.to_csv('../2021/old_allplayers.csv', index=False)